In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data reading and discovering

In [ ]:
data=pd.read_csv('La myriade de Totems de Montpellier - SaisiesFormulaire.csv', sep = ',', encoding= 'utf_8')

In [ ]:
########################
# delete empty columns #
########################

del data['Unnamed: 4']
del data['Remarque']

###################
# displaying data # 
###################

display(data)

In [ ]:
data.describe()

In [ ]:
print(data.columns)
display(data.head())

In [ ]:
print(data.isnull().sum())

In [ ]:
data[data.isna().any(axis=1)]

In [ ]:
dataset=data.dropna()
dataset['Date']= pd.to_datetime(dataset['Date'] + ' ' + dataset['Heure / Time'], format="%d/%m/%Y %H:%M:%S")
dataset=dataset.drop('Heure / Time',axis=1)

In [ ]:
dataset

In [ ]:
df=dataset.loc[:,["Date","Vélos ce jour / Today's total"]]
df.columns= ["Date","TotalPerDay"]

In [ ]:
df=df.groupby('Date')['TotalPerDay'].sum().reset_index()
#df[sub.Date=='12/03/2020'] to verify

In [ ]:
#plot
df.plot.bar(figsize=(70,7),rot= 70, x='Date',y='TotalPerDay')

we notice that the max nb of bikes per day is around 16000  

we also notice that bikes pass the most in June, July, August and March. To confirm that, we visualise it as following :

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None) #
display(df)

In [ ]:
df1=df.groupby([(df.Date.dt.year),(df.Date.dt.month)])['TotalPerDay'].sum()
display(df1)
df1.plot.bar()

In [ ]:
dataset_traitement=dataset
dataset_traitement.columns
data_train=dataset_traitement[["Date","Vélos ce jour / Today\'s total"]]
data_train.rename(columns={ "Vélos ce jour / Today\'s total": "y"})
data_train.sort_values(by=['Date'], inplace=True, ascending=True)
hour_df = data_train.resample('1min', on='Date').mean()
w_df = hour_df.reset_index().dropna()
w_df.columns = ['ds', 'y']


In [ ]:
from fbprophet import Prophet

#P=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=False, changepoint_range=1) 
#interval_width sets the uncertainty interval to produce a confidence interval around the forecast
#P.add_seasonality(name='monthly', period=30.5, fourier_order=5, prior_scale=0.02)
P=Prophet()
P.add_country_holidays(country_name='France')
P.fit(w_df)


#future = P.make_future_dataframe(freq='W', periods=4)  # Let's predict the next month's average prices

#future.tail()

In [ ]:
# define the period for which we want a prediction
future = list()
for i in range(12, 15):
  for j in range(0,60):
	  date = "2021-03-16 " +str(i)+":"+str(j)
	  future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= pd.to_datetime(future['ds'])
# use the model to make a forecast
forecast = P.predict(future)
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

